## Import libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from ipywidgets import interact, widgets

In [ ]:
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

# Novel samples: prediction of novel ecosystems

In [ ]:
# Interface to ask for age, temperature and precipitations3Days (with unit and valid range) 
# https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#IntSlider

a = widgets.FloatSlider(min=40,max=100,value=60)
b = widgets.FloatSlider(min=0,max=5,value=1.5)
c = widgets.IntSlider(min=1,max=20,value=4)


accordion = widgets.Accordion(children=[a,b,c])
accordion.set_title(0, 'Temperature (ºF):')
accordion.set_title(1, 'Precipitations 3 days (inches):')
accordion.set_title(2, 'Plant age (weeks):')
accordion

#out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

t = widgets.Label(value="Please, select the environmental features associated to the maize plant/soil to what you would like to predict its rhizosphere microbiome:")

widgets.VBox([t, accordion])





In [ ]:
temp=a.value
rain=b.value
age=c.value

In [ ]:
metadata = pd.DataFrame({'X.SampleID': ['new01'], 'age': [age], 'Temperature': [temp], 'Precipitation3Days':[rain]}, 
                  columns=['X.SampleID','age','Temperature','Precipitation3Days'])
metadata = metadata.set_index('X.SampleID')
domain = metadata[['age',
                   'Temperature',
                   'Precipitation3Days']]
domain

In [ ]:
metadata.shape

In [ ]:
domain_novel_samples = domain.to_numpy(dtype=np.float32)

#### Load models

In [ ]:
decoder = tf.keras.models.load_model('../Results/Models/decoder.h5')
encoder_domain = tf.keras.models.load_model('../Results/Models/encoder_domain_model_latent.h5')

#### Compute new predictions

In [ ]:
# Input only domain (i.e. environmental features)
pred_latent_novel_samples = encoder_domain.predict(domain_novel_samples)
pred_domain_novel_samples = decoder.predict(pred_latent_novel_samples)

In [ ]:
def save_predicted_otu_table(pred,sample_names,otu_names,suffix=''):
    df_otu = pd.DataFrame(pred, index=sample_names, columns=otu_names)
    df_otu.T.to_csv('../Results/otus_'+suffix+'.tsv', index=True, header=True, sep='\t')
  
    return df_otu

In [ ]:
# Get otu IDs
otu = pd.read_csv('../Datasets/otu_table_all_80.csv', index_col=0, header=None, sep='\t').T
otu = otu.set_index('otuids')
otuids = otu.columns

In [ ]:
otuids

In [ ]:
df_pred_otu = save_predicted_otu_table_and_latent(pred_domain_novel_samples,domain.index,otuids,'predFromDomain_novelSamples_byHandAPIvalues')

In [ ]:
pred_domain_novel_samples.shape

In [ ]:
pred_domain_novel_samples

In [ ]:
### TO-DO: call R script to plot some phylosep plot
### To get phyloseq object, taking code from: https://github.com/jorgemf/DeepLatentMicrobiome/blob/master/Src/plot_novel_samples_predictions_radarchart.r
#file_tax = '../Datasets/tax_table_all_80_cleanNames.csv'
#physeq_pred = build_physeq_object('../Results/otus_predFromDomain_novelSamples_byHandAPIvalues.tsv','../Datasets/metadata_novel_samples_byHandAPIvalues_only3envFeatures.csv',file_tax)
